In [1]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from itertools import tee, chain, islice
from math import log
from operator import itemgetter
from itertools import takewhile
import urllib
import requests
import time

In [2]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"
filesInYearPattern = 'judgments-(316[3-9]|317\d)\.json'

In [3]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items']
    year_filtered = filter(lambda item: year in item['judgmentDate'],
                           judgements)
    yield from map(lambda item: (item['textContent'], item['judgmentDate']),
                   year_filtered)

In [4]:
def clean_text(line):
    notags = re.sub(r"<[^>]*>", " ", line)
    nobreaks = re.sub(r"-\n", " ", notags)
    noparens = re.sub("\(\.\.\.\)", " ", nobreaks)
    nonewline = re.sub("\\n", " ", noparens)
    return nonewline.replace(u'\xa0', u' ')

In [6]:
json_files = os.listdir(json_data_dir)
judgements_files = filter(lambda name: re.match(filesInYearPattern, name),
                          json_files)
text_date = list(chain.from_iterable(map(judgement_texts, judgements_files)))
to_process = sorted(text_date, key=itemgetter(1))[:100]
texts = [clean_text(t[0]) for t in to_process]

bytearray(b'         UZASADNIENIE         Pow\xc3\xb3d     Sp. z o.o.  Sp. k. z siedzib\xc4\x85 w  P.  wni\xc3\xb3s\xc5\x82 o zwolnienie spod egzekucji administracyjnej prawa maj\xc4\x85tkowego przys\xc5\x82uguj\xc4\x85cego powodowi w postaci wierzytelno\xc5\x9bci z tytu\xc5\x82u nienale\xc5\xbcnie pobranego podatku akcyzowego za miesi\xc4\x85ce luty, marzec, kwiecie\xc5\x84, maj, wrzesie\xc5\x84 i pa\xc5\xbadziernik 2011 r. okre\xc5\x9blonego w decyzji Naczelnika Urz\xc4\x99dy Celnego w  Z.  z dnia 6 lutego 2013 r. nr        -   9110-414/12/13/DN oraz Dyrektora Izby Celnej w  R.  z dnia 8 kwietnia 2014 r. nr        -   -24/13/14, kt\xc3\xb3ra zosta\xc5\x82 stwierdzona prawomocnym wyrokiem Wojew\xc3\xb3dzkiego S\xc4\x85du Administracyjnego w  G.  z dnia 6 lipca 2016 r. sygn. akt I SA/Go 186/16 w postaci koszt\xc3\xb3w egzekucyjnych w kwocie 8.614,98 z\xc5\x82otych wynikaj\xc4\x85cych z egzekucji ww. nienale\xc5\xbcnie pobranego podatku akcyzowego. Wierzytelno\xc5\x9b\xc4\x87 ta zosta\x

In [7]:
url = "http://ws.clarin-pl.eu/nlprest2/base"
json_data = {'lpmn': lpmn, 'user': user, 'text': str(texts[0].encode('utf-8'),'utf-8')}
#r = urllib2.Request(,doc,{'Content-Type': 'binary/octet-stream'})
resp = requests.post(
    url + '/process/',
    data=json.dumps(json_data),
    headers={'Content-Type': 'application/json'})

NameError: name 'lpmn' is not defined

In [ ]:
def upload(text):
    doc = to_binary(text)
    return requests.post(
        url + '/upload/',
        data=doc,
        headers={
            'Content-Type': 'binary/octet-stream'
        }).text


def process(data):
    doc = json.dumps(data)
    taskid = requests.get(
        url + '/startTask/',
        headers={
            'Content-Type': 'application/json'
        },
        data=doc).text
    time.sleep(0.2)
    resp = requests.get(url + '/getStatus/' + taskid).text
    print(resp)
    data = json.loads(resp)
    while data["status"] == "QUEUE" or data["status"] == "PROCESSING":
        time.sleep(0.5)
        resp = requests.get(url + '/getStatus/' + taskid).text
        data = json.loads(resp)
    if data["status"] == "ERROR":
        print("Error " + data["value"])
        return None
    return data["value"]

In [ ]:
fileid = upload(texts[0])
lpmn = "any2txt|wcrft2"
user = "mojadresemail"
out_path = 'out/'
data = {'lpmn': lpmn, 'user': user, 'file': fileid}
data = process(data)
if data == None:
    continue
data = data[0]["fileID"]
content = urllib2.request.urlopen(
    urllib2.Request(url + '/download' + data)).read()
with open(out_path + os.path.basename(file) + '.ccl', "w") as outfile:
    outfile.write(content)